<a href="https://colab.research.google.com/github/Ikeda33/codespaces-railstutorial/blob/main/cycleGAN_local_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ローカルのGPUに繋げる
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8889 --NotebookApp.port_retries=0 --NotebookApp.iopub_data_rate_limit 10000000

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import argparse
import glob
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm
import torchvision.transforms as transforms
import torchvision.utils as vutils

from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epoch", type = int, default=50)
parser.add_argument("--batch_size", type = int, default=1)
parser.add_argument("--lr", type = float, default=3e-4)
parser.add_argument("--decay_start", type = int, default=100)
parser.add_argument("--weight_identity", type = float, default=5.0)
parser.add_argument("--weight_cycle", type = float, default=10.0)

parser.add_argument("--image_size", type = int, default=256)
parser.add_argument("--n_ch", type = int, default=64)

parser.add_argument("--z_dim", type = int, default=100)
parser.add_argument("--beta1", type = float, default=0.5)

opt = parser.parse_args(args=[])
print(opt)

Namespace(n_epoch=50, batch_size=1, lr=0.0003, decay_start=100, weight_identity=5.0, weight_cycle=10.0, image_size=256, n_ch=64, z_dim=100, beta1=0.5)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#localの時は消す

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
class ImageDataset(Dataset):
    def __init__(self,  transform = None):
        super().__init__()
        #self.files_A = glob.glob("/content/drive/MyDrive/horse2zebra (1)/horse2zebra/trainA/*.jpg")#colab
        #self.files_B = glob.glob("/content/drive/MyDrive/horse2zebra (1)/horse2zebra/trainB/*.jpg")#colab
        self.files_A = glob.glob(R"C:\Users\ikeda\Downloads\horse2zebra (1)\horse2zebra\trainA\*.jpg")#local "C:\Users\ikeda\Downloads\horse2zebra (1)\horse2zebra\trainA\n02381460_2.jpg"
        self.files_B = glob.glob(R"C:\Users\ikeda\Downloads\horse2zebra (1)\horse2zebra\trainB\*.jpg")#local

        self.transform = transform
    
    def __getitem__(self, index):
      imgA = self.transform(Image.open(self.files_A[index % len(self.files_A)]))
      while True:
        random_index = np.random.randint(0, len(self.files_B) - 1)
        imgB = self.transform(Image.open(self.files_B[random_index % len(self.files_B)]))
        C, H, W = imgB.size()
        if C == 3:
          break
      return {"A":imgA, "B":imgB}

    
    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [ ]:
class DecayLR(object):
  def __init__(self, n_epoch, offset, decay_start_epoch):
    self.n_epoch = n_epoch
    self.offset = offset
    self.decay_start_epoch = decay_start_epoch

  def step(self, epoch):
    return 1.0 - max(0, epoch + self.offset -self.decay_start_epoch)/(self.n_epoch - self.decay_start_epoch)

In [ ]:
class ReplayBuffer:
  def __init__(self, max_size = 50):
    self.max_size = max_size
    self.data = []

  def push_and_pop(self, data):
    to_return = []
    for element in data.data:
      element = torch.unsqueeze(element, 0)
      if len(self.data) < self.max_size:
        self.data.append(element)
        to_return.append(element)
      else:
        if np.random.rand() > 0.5:
          i = np.random.randint(0, self.max_size - 1)
          to_return.append(self.data[i].clone())
          self.data[i] = element
        else:
          to_return.append(element)
    return torch.cat(to_return)
    

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.conv_layers = nn.Sequential(
        nn.ReflectionPad2d(1),
        nn.Conv2d(in_channels, in_channels, 3),
        nn.InstanceNorm2d(in_channels),
        nn.ReLU(inplace = True),
        nn.ReflectionPad2d(1),
        nn.Conv2d(in_channels, in_channels, 3),
        nn.InstanceNorm2d(in_channels),
        
    )
  def forward(self, x):
    out = self.conv_layers(x)
    out = out + x
    return out

In [ ]:
class Generator(nn.Module):
  def __init__(self, res_block, in_channels = 3, n_ch = opt.n_ch):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.ReflectionPad2d(3),
        nn.Conv2d(in_channels,n_ch, 7),
        nn.InstanceNorm2d(n_ch),
        nn.ReLU(inplace=True),

        nn.Conv2d(n_ch,n_ch * 2, 3, stride = 2, padding = 1),
        nn.InstanceNorm2d(n_ch * 2),
        nn.ReLU(inplace=True),

        nn.Conv2d(n_ch * 2,n_ch * 4, 3, stride = 2, padding = 1),
        nn.InstanceNorm2d(n_ch * 4),
        nn.ReLU(inplace=True),    
    )
    #transformer
    self.res_block = res_block(n_ch * 4)
    self.transformer = nn.ModuleList([
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4), 
            res_block(n_ch * 4),                     
    ])
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(n_ch * 4, n_ch * 2, 3, stride = 2, padding = 1, output_padding=1),
        nn.InstanceNorm2d(n_ch * 2),
        nn.ReLU(inplace=True),

        nn.ConvTranspose2d(n_ch * 2, n_ch , 3, stride = 2, padding = 1, output_padding=1),
        nn.InstanceNorm2d(n_ch),
        nn.ReLU(inplace=True),

        nn.ReflectionPad2d(3),
        nn.Conv2d(n_ch, 3, 7),
        nn.Tanh()
    )

  def forward(self, x):
    out = self.encoder(x)
    for func in self.transformer:
      out = func(out)
    out = self.decoder(out)
    return out

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, n_ch = opt.n_ch):
    super().__init__()
    self.conv1 = self.conv_layer(3, n_ch, 4, 2, 1)
    self.conv2 = self.conv_layer(n_ch, n_ch*2, 4, 2, 1)
    self.conv3 = self.conv_layer(n_ch*2, n_ch*4, 4, 2, 1)
    self.conv4 = self.conv_layer(n_ch*4, n_ch*8, 4, 1, 1)
    self.conv5 = nn.Conv2d(n_ch*8, 1,   4, 1,padding = 1)

  @staticmethod
  def conv_layer(in_channels, out_channels, kernel_size, stride, padding, has_norm= True):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)]
    if has_norm == True:
      layers.append(nn.InstanceNorm2d(out_channels))
    layers.append(nn.LeakyReLU(0.2, inplace =True))
    net = nn.Sequential(*layers)
    return net

  def forward(self, x):
    out = self.conv1(x)
    out = self.conv2(out)
    out = self.conv3(out)
    out = self.conv4(out)
    out = self.conv5(out)
    B, C, H, W = out.size()
    out = F.avg_pool2d(out, (H, W))# squeeze->imagesize = 1*1
    out = out.view(B, -1)#-> 1dimension
    return out

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        

In [ ]:

from torchvision.transforms.transforms import CenterCrop

transform = transforms.Compose([
    
    transforms.Resize(int(opt.image_size * 1.12), Image.BICUBIC),#Image.BICUBIC は画像の互換法
    transforms.RandomCrop(opt.image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), 
    transforms.Normalize((0.5,),(0.5,))])

C:\Users\ikeda\anaconda3\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [ ]:
dataset = ImageDataset(transform = transform)

In [ ]:
dataloader = DataLoader(dataset = dataset, batch_size = opt.batch_size,num_workers = 2,  shuffle = True)#データの読み込み

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
netG_A2B = Generator(ResidualBlock).to(device)
netG_A2B.apply(weights_init)

netG_B2A = Generator(ResidualBlock).to(device)
netG_B2A.apply(weights_init)

netD_A = Discriminator().to(device)
netD_A.apply(weights_init)

netD_B = Discriminator().to(device)
netD_B.apply(weights_init)


Discriminator(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): LeakyReLU(negative_slope=0.2, 

In [ ]:
adversarial_loss = torch.nn.MSELoss().to(device)
cycle_loss = torch.nn.L1Loss().to(device)
identity_loss = torch.nn.L1Loss().to(device)

In [ ]:
optimizer_D_A = optim.Adam(netD_A.parameters(), lr = opt.lr, betas = (opt.beta1, 0.999))
optimizer_D_B = optim.Adam(netD_B.parameters(), lr = opt.lr, betas = (opt.beta1, 0.999))

optimizer_G = optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr = opt.lr, betas = (opt.beta1, 0.999))#二つの生成器を同時に最適化


In [ ]:
lr_lambda = DecayLR(opt.n_epoch, 0, opt.decay_start).step
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda = lr_lambda)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda = lr_lambda)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda = lr_lambda)


In [ ]:
"""#出力のためのディレクトリ
import os

model_name = "CycleGAN"
f_path_params = "/content/drive/MyDrive/params/{}".format(model_name)
f_path_result  = "/content/drive/MyDrive/result/{}".format(model_name)

os.makedirs(f_path_result, exist_ok = True)
os.makedirs(f_path_params, exist_ok = True)
"""
#colab

'#出力のためのディレクトリ\nimport os\n\nmodel_name = "CycleGAN"\nf_path_params = "/content/drive/MyDrive/params/{}".format(model_name)\nf_path_result  = "/content/drive/MyDrive/result/{}".format(model_name)\n\nos.makedirs(f_path_result, exist_ok = True)\nos.makedirs(f_path_params, exist_ok = True)\n'

In [ ]:
"""#パラメータ保存
def save_params(epoch, dir_path, model_list, model_name_list):
  for model, model_name in zip(model_list, model_name_list):
    file_path = dir_path + "/{model}_{epoch}.pth".format(model = model_name, epoch = epoch)
    torch.save(model.state_dict(), file_path)"""
  #colab

'#パラメータ保存\ndef save_params(epoch, dir_path, model_list, model_name_list):\n  for model, model_name in zip(model_list, model_name_list):\n    file_path = dir_path + "/{model}_{epoch}.pth".format(model = model_name, epoch = epoch)\n    torch.save(model.state_dict(), file_path)'

In [ ]:
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

for epoch in range(0, opt.n_epoch):
  running_loss_D = 0.0
  running_loss_G = 0.0
  running_loss_G_GAN = 0.0
  running_loss_G_cycle = 0.0
  running_loss_G_identity = 0.0

  for data in tqdm.tqdm(dataloader, position = 0):

    real_img_A = data["A"].to(device)
    real_img_B = data["B"].to(device)

    batch_size = real_img_A.size()[0]

    real_label = torch.ones([batch_size, 1]).to(device)#use generator loss
    fake_label = torch.zeros([batch_size, 1]).to(device)

    #train_generator
    optimizer_G.zero_grad()#勾配初期化
    #adversarial loss
    fake_img_A = netG_B2A(real_img_B)
    fake_img_B = netG_A2B(real_img_A)

    output_A = netD_A(fake_img_A)
    output_B = netD_B(fake_img_B)

    loss_GAN_A2B = adversarial_loss(output_B, real_label)#本物と判別されたいから ->real_label
    loss_GAN_B2A = adversarial_loss(output_A, real_label)
    #cycle loss
    cycle_img_A = netG_B2A(fake_img_B)
    cycle_img_B = netG_A2B(fake_img_A)

    loss_cycle_ABA = cycle_loss(cycle_img_A, real_img_A)
    loss_cycle_BAB = cycle_loss(cycle_img_B, real_img_A)
    #identity loss
    identity_img_A = netG_B2A(real_img_A)
    identity_img_B = netG_A2B(real_img_B)

    loss_identity_A = identity_loss(identity_img_A, real_img_A)
    loss_identity_B = identity_loss(identity_img_B, real_img_B)

    lossG = (loss_GAN_A2B +loss_GAN_B2A + 
             opt.weight_cycle * (loss_cycle_ABA +loss_cycle_BAB) + 
             opt.weight_identity *  (loss_identity_A + loss_identity_B))
    
    lossG.backward()
    optimizer_G.step()

    #train discriminator
    optimizer_D_A.zero_grad()
    optimizer_D_B.zero_grad()
#実際の画像から損失を計算
    real_output_A = netD_A(real_img_A)
    real_output_B = netD_B(real_img_B)

    loss_DA_real = adversarial_loss(real_output_A, real_label)
    loss_DB_real = adversarial_loss(real_output_B, real_label)
    #偽物の画像から損失を計算する
    fake_img_A  = fake_A_buffer.push_and_pop(fake_img_A)#5０枚のバッファを超えてくると確率的に古い偽物画像が送られる
    fake_img_B  = fake_B_buffer.push_and_pop(fake_img_B)

    fake_output_A = netD_A(fake_img_A.detach())
    fake_output_B = netD_B(fake_img_B.detach())

    loss_DA_fake = adversarial_loss(fake_output_A, fake_label)
    loss_DB_fake = adversarial_loss(fake_output_B, fake_label)

    loss_DA = (loss_DA_real +loss_DA_fake )*0.5
    loss_DB = (loss_DB_real +loss_DB_fake )*0.5

    loss_DA.backward()
    loss_DB.backward()
    optimizer_D_A.step()
    optimizer_D_B.step()
    #adding to running loss
    running_loss_D += (loss_DA.item() + loss_DB.item())/2.0
    running_loss_G += lossG.item()

    running_loss_G_GAN += (loss_GAN_A2B.item() + loss_GAN_B2A.item()) / 2.0
    running_loss_G_cycle += (loss_cycle_ABA.item() + loss_cycle_BAB.item()) / 2.0
    running_loss_G_identity += (loss_identity_A.item() + loss_identity_B.item()) / 2.0
  lr_scheduler_G.step()
  lr_scheduler_D_A.step()
  lr_scheduler_D_B.step()
  running_loss_D /= len(dataloader)
  running_loss_G /= len(dataloader)
  running_loss_G_GAN/= len(dataloader)
  running_loss_G_cycle /= len(dataloader)
  running_loss_G_identity/= len(dataloader)
  loss_log = """
  epoch:{},
  loss_D:{},
loss_G :{},
loss_G_GAN:{},
loss_G_cycle :{},
loss_G_identity:{},
   """.format(epoch, running_loss_D,running_loss_G,
              running_loss_G_GAN, running_loss_G_cycle, running_loss_G_identity)
  print(loss_log)
  fake_imgs = torch.cat([fake_img_A, fake_img_B])
  grid_imgs = vutils.make_grid(fake_imgs.detach())
  grid_imgs_arr = grid_imgs.cpu().numpy()
  plt.imshow(np.transpose(grid_imgs_arr, (1,2,0)))
  plt.show()
 # model_list = [netG_A2B, netG_B2A, netD_A, netD_B]
  #model_name_list = ["netG_A2B", "netG_B2A", "netD_A", "netD_B"]
  #save_params(epoch, f_path_params, model_list, model_name_list)

  0%|                                                                                         | 0/1334 [00:00<?, ?it/s]